In [1]:
import json
import time

from PIL import Image
from io import BytesIO
from base64 import b64decode as decoder
from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager

In [2]:
class Spider:
    """
    This is the constructor class to which you can pass a bunch of parameters. 
    These parameters are stored to the class instance variables so that the
    class functions can access them later.
    
    no_of_pages: number of pages to scrape
    """
    def __init__(self, no_of_pages=1):
        self.driver = webdriver.Firefox(
            executable_path=GeckoDriverManager().install()
        )
        self.no_of_pages = no_of_pages
        self.current_page = dict()
                
    def get_raw_page(self, page_rules, page_id=1):
        assert type(page_rules) == dict
        rules = dict()
        result = dict()
        driver = self.driver

        
        for key, value in page_rules.items():
            rules[key] = value.format(page_id)

        url = rules['url']
        driver.get(url)
        time.sleep(2)

        result[rules['input_tag']] = self.get_target(
                target = rules['target'],
                css_selector = rules['css_selector']
            )
            
        self.current_page = result
        return result          
    
    def get_target(self, target='', css_selector=''):
        '''
        Get target from DOM
        '''
        
        driver = self.driver
                
        if target == 'src':
            try:
                return driver.find_element_by_css_selector(
                    css_selector
                ).get_attribute('src')
            except:
                return None
        else:
            return None

    def save_page(self, page_rules, page_id=1):
        assert type(page_rules) == dict
        current_page = self.current_page
        
        if current_page == {}:
            get_raw_page(page_rules, page_id)
        
        img_raw = current_page['img-base64'].replace("data:image/jpg;base64,","")
        img = Image.open(BytesIO(decoder(img_raw)))
        img.save('img/page-{}.jpg'.format(page_id),'JPEG')
        
        print('...Image saved...')

In [3]:
my_spider = Spider()

[WDM] - ====== WebDriver manager ======


[WDM] - Driver [/home/tonalli/.wdm/drivers/geckodriver/linux64/v0.29.0/geckodriver] found in cache


In [4]:
page_rules = {
        'url': 'http://appstrillas.mx/pdfFlipping/viewer.jsp?id=SbS3AB#page/{}',
        'input_tag': 'img-base64',
        'output_tag': 'page-{:03}',
        'target': 'src',
        'css_selector': '.p{} > img:nth-child(2)',
}

In [5]:
page_id=77

In [7]:
current_page = my_spider.get_raw_page(page_rules, page_id)

In [8]:
my_spider.save_page(page_rules, page_id)

...Image saved...
